In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd 
import numpy as np
import json
import copy
import time
from datetime import datetime 

from api_wrapper import alph_settings as alph, val_steps as val, alph_api_wrapper as wrap, threading
from preproc import filters, make_dfs as df

In [5]:
settings = alph(
    apikey = "MD27QTTVAK3AJBUQ", 
    site = "https://www.alphavantage.co/query?", 
    date_range = [datetime(2016, 8, 9), datetime(2022, 8, 9)]
    )

In [4]:
#get tickers that are valid across the past six years
tker_dict = val.get_tkers(settings)
valid_lst = val.get_valid_tkers(tker_dict, pretty_print = False)

In [6]:
#filtering the tickers we want for analysis 
raw_df = pd.DataFrame(valid_lst, columns = ["ticker", "name", "exchange", "type", "date", "delist", "status"])
type_filt = "Stock" 
exchange_filt = "NYSE"
raw_df = raw_df.query("type == @type_filt")
raw_df = raw_df.query("exchange == @exchange_filt")
display(raw_df)

#list of tickers to work on for now
ticker_lst = raw_df["ticker"].tolist()
#test_lst = raw_df["ticker"].tolist()[:1000]

,ticker,name,exchange,type,date,delist,status
0,DAN,Dana Inc,NYSE,Stock,2008-01-02,null,Active
8,PEAK,Healthpeak Properties Inc,NYSE,Stock,1987-11-05,null,Active
16,MX,MagnaChip Semiconductor Corp,NYSE,Stock,2011-03-11,null,Active
17,SCX,L.S. Starrett Co. - Class A,NYSE,Stock,1987-12-30,null,Active
18,SLG-P-I,SL Green Realty Corporation Preferred Series I,NYSE,Stock,2002-07-01,null,Active
...,...,...,...,...,...,...,...
5574,CMP,Compass Minerals International Inc,NYSE,Stock,2003-12-12,null,Active
5581,JNPR,Juniper Networks Inc,NYSE,Stock,1999-06-25,null,Active
5583,BAP,Credicorp Ltd,NYSE,Stock,1995-10-25,null,Active
5584,ENB,Enbridge Inc,NYSE,Stock,1990-03-27,null,Active


In [ ]:
#get cofi and filter out bad tickers 
cofi_lst = ['MarketCapitalization', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta']
hit_lst = copy.deepcopy(ticker_lst)
time_it = 3
ticker_info_data = {}
while len(hit_lst) > 0 and time_it > 2:
    ticker_info_url = wrap.get_general_url(settings, ticker_lst = hit_lst, output = "lst", datatype = "csv", function = "OVERVIEW")
    hit_lst, time_it, data = threading.thrd_json_data(ticker_info_url, key_lst = cofi_lst, wrapper_function = wrap.get_json_data)
    ticker_info_data.update(data)
    print(hit_lst)
    time.sleep(10)
with open(r'C:\Users\ELIZABETH CHENG\MLAP_Proj\pairs_selection\reader\NYSE_cofi.json', 'w') as fp:
    json.dump(ticker_info_data, fp)

In [11]:
#filter out tickers with missing data
cofi_filter, filtered_cofi_dict = filters.cofi_filter(ticker_info_data)
clean_df = raw_df.query("ticker != @cofi_filter")
display(clean_df)

,ticker,name,exchange,type,date,delist,status
0,DAN,Dana Inc,NYSE,Stock,2008-01-02,null,Active
8,PEAK,Healthpeak Properties Inc,NYSE,Stock,1987-11-05,null,Active
16,MX,MagnaChip Semiconductor Corp,NYSE,Stock,2011-03-11,null,Active
19,CRS,Carpenter Technology Corp,NYSE,Stock,1987-11-05,null,Active
28,PWR,Quanta Services Inc,NYSE,Stock,1998-02-12,null,Active
...,...,...,...,...,...,...,...
5574,CMP,Compass Minerals International Inc,NYSE,Stock,2003-12-12,null,Active
5581,JNPR,Juniper Networks Inc,NYSE,Stock,1999-06-25,null,Active
5583,BAP,Credicorp Ltd,NYSE,Stock,1995-10-25,null,Active
5584,ENB,Enbridge Inc,NYSE,Stock,1990-03-27,null,Active


In [60]:
temp_lst = clean_df["ticker"].tolist()

In [ ]:
#get daily data 
hit_lst_2 = copy.deepcopy(temp_lst)
time_it_2 = 3
daily_data = {}
while len(hit_lst_2) > 0 and time_it_2 > 2:
    daily_urls = wrap.get_general_url(settings, datatype = "csv", ticker_lst = hit_lst_2, output = "lst", function = "TIME_SERIES_DAILY_ADJUSTED")
    hit_lst_2, time_it_2, data = threading.thrd_csv_data(daily_urls)
    daily_data.update(data)
    print(hit_lst_2)
    time.sleep(15)
with open(r'C:\Users\ELIZABETH CHENG\MLAP_Proj\pairs_selection\reader\NYSE_daily_data.json', 'w') as fp:
    json.dump(daily_data, fp)

In [ ]:
start = settings.date_range[0] 
end = settings.date_range[1]
data_dict = {i: [[j[0], float(j[5]), int(j[6])] for j in daily_data[i][1:]] for i in daily_data.keys()}
data_key = list(data_dict.keys())
for i in data_key: 
    data_dict[i] = pd.DataFrame(data_dict[i][1+1:], columns = [daily_data[i][0][0], daily_data[i][0][5], daily_data[i][0][6]])
    data_dict[i]["timestamp"] = pd.to_datetime(data_dict[i]["timestamp"])
    data_dict[i] = data_dict[i].query("@start <= timestamp <= @end")
    if len(data_dict[i]) != 1511: 
        print(i, len(data_dict[i]))
        data_dict.pop(i)

In [ ]:
#get df for pca (price, volume)
price_pivot = df.pivot_data(data_dict, key = "adjusted_close")
vol_pivot = df.pivot_data(data_dict, key = "volume")
display(price_pivot)

price_pivot.to_csv(r'C:\Users\ELIZABETH CHENG\source\repos\MLAP\reader\price_pca.csv')
vol_pivot.to_csv(r'C:\Users\ELIZABETH CHENG\source\repos\MLAP\reader\vol_pca.csv')

In [41]:
with open(r'C:\Users\ELIZABETH CHENG\source\repos\MLAP\reader\NYSE_cofi.json', 'r') as fp:
    cofi_dict = json.load(fp)

In [47]:
#get df for pca (cofi)
bad_lst, cofi_clean = filters.cofi_filter(cofi_dict)
cofi_pivot = df.pivot_cofi(cofi_clean)
display(cofi_pivot)

#write to reader
cofi_pivot.to_csv(r'C:\Users\ELIZABETH CHENG\source\repos\MLAP\reader\cofi_pca.csv')

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
0,,,,,,,,,,,,,,,,,,,,,
PEAK,1.244813e+10,1.20,0.0522,0.40,3.674,0.1130,0.1730,0.0145,0.0326,1.980974e+09,...,0.40,-0.754,0.088,30.33,57.670,6.2800,1.902,9.380,17.020,0.823
MX,4.292790e+08,0.00,0.0000,1.50,9.660,0.1590,0.1130,0.0549,0.1620,4.428040e+08,...,1.50,-0.257,-0.110,21.00,6.370,0.9700,0.963,0.364,1.560,1.133
PWR,2.006041e+10,0.27,0.0020,3.06,108.540,0.0292,0.0432,0.0384,0.0961,1.547434e+10,...,3.06,-0.272,0.411,151.67,45.840,1.2960,3.722,1.491,17.670,1.095
DAN,2.297949e+09,0.40,0.0283,0.67,66.200,0.0103,0.0227,0.0174,0.0515,9.543000e+09,...,0.67,-0.845,0.173,15.00,23.930,0.2410,1.050,0.469,6.460,2.268
NPO,2.152539e+09,1.10,0.0111,8.61,59.220,0.1460,0.1030,0.0320,0.1440,1.225900e+09,...,8.61,0.128,0.116,128.00,12.020,1.7560,1.546,2.243,8.160,1.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EXK,6.655980e+08,0.00,0.0000,0.07,0.990,-0.0304,0.1180,0.0403,-0.0199,1.716010e+08,...,0.07,-0.029,-0.356,5.18,50.140,3.8790,2.302,3.143,14.740,1.484
DLNG,1.104070e+08,0.00,0.1490,1.43,3.720,0.4610,0.4250,0.0373,0.1640,1.370480e+08,...,1.43,0.220,-0.015,4.00,2.098,0.8060,0.273,4.019,4.832,1.649
TPC,3.600180e+08,0.00,0.0345,0.98,78.780,-0.0099,0.0027,0.0014,-0.0084,4.028173e+09,...,0.98,-0.173,-0.294,10.50,7.150,0.0894,0.219,0.249,9.460,1.367


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#PCA logic flow 

#Step 1: get the standardised features
x = df (with the features from the dataframe)
# Standardizing the features
x = StandardScaler().fit_transform(x)

#Step 2: feeed standardadised features into the PCA
#We can experiment with different PCA n later? 
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ["pc1", "pc2"])

#Step 3: Check if explained variance is good enough
print(pca.explained_variance_ratio_)

#Step 4: 
Visualisation for slides idk 